In [76]:
import pandas as pd
import json
import os
from tqdm import tqdm
import pickle

gather a list of failed video ids
 - api fail
 - no video detail
create two JSON text files holding all successful video details and comments
 - videoID as keys
 - some videos disabled comments so n_video in video details will be more than n_video in comment file

In [3]:
# read in api failure list 
fp = '../data/raw/api_data/failed_lookups.txt'

with open(fp) as f:
    failed_dict = json.load(f)
    
failed_dict.keys()

dict_keys(['failed'])

In [60]:
failed_ids = set([lst[0] for lst in failed_dict['failed']]) #273

In [9]:
# read in test_batch 
test_batch = '../data/processed/test_batch.txt'

with open(test_batch) as tb:
    test_files = json.load(tb)

test_files.keys()

dict_keys(['HTft', 'VX3C', 'j2Er', '3HOO', '6nfW', 'NX6e', 'ljAU', 'zmk6', 'qX7a', '3Z4A', 'kugn', '2D9o', 'Tvnd', 'A9Jp', 'f3CR'])

In [63]:
def get_videoDetails(data, report = False):

    # for every yt8M id, check if videoDetail is available
    # if not, check if it's in failure list, if not, add to failure list
    # if it is, append to buffer list 
    
    buffer = []
    n_failed = 0
    for idx in tqdm(data.keys()):
        if data[idx]['details'] == 'Invalid lookup.':
            failed_ids.add(idx) 
            n_failed += 1
        elif len(data[idx]['details']['items']) == 0:
            failed_ids.add(idx)
            n_failed += 1
        else:
            data[idx]['details']['items'][0]['yt8M_id'] = idx
            buffer.append(data[idx]['details']['items'][0])
    
    # json_normalize buffer list into videoDetails df
    videoDets_df = pd.json_normalize(buffer, errors='ignore')
    
    # print stats for batch
    if report:
        print(f'n of videos successful: {videoDets_df.shape[0]}')
        print(f'n of videos failed: {n_failed}')
    
    return videoDets_df
    

In [64]:
test_df = get_videoDetails(test_files, report=True)
test_df.head()

100%|████████████████████████████████████████| 15/15 [00:00<00:00, 65196.44it/s]

n of videos successful: 15
n of videos failed: 0


,kind,etag,id,yt8M_id,snippet.publishedAt,snippet.channelId,snippet.title,snippet.description,snippet.thumbnails.default.url,snippet.thumbnails.default.width,...,status.embeddable,status.publicStatsViewable,status.madeForKids,statistics.viewCount,statistics.likeCount,statistics.favoriteCount,statistics.commentCount,player.embedHtml,topicDetails.topicCategories,recordingDetails.recordingDate
0,youtube#video,qw6REl1mcyQq7UoW1-Dkm1yCrOY,130S1TN3P0M,HTft,2014-11-05T07:49:51Z,UCh_KdtEAc7qwyeXyVuuYPUA,Obserwator 22: Jesień na tamie / Autumn on a ...,To już pożegnanie z tamą bobrową. Jesień zawit...,https://i.ytimg.com/vi/130S1TN3P0M/default.jpg,120,...,True,True,False,2415,19,0,3,"<iframe width=""480"" height=""360"" src=""//www.yo...",[https://en.wikipedia.org/wiki/Lifestyle_(soci...,NaN
1,youtube#video,hBay_xFV_egFO5I_5lVHS8xcSi4,iHpgw8WN6Bg,VX3C,2011-11-01T06:43:23Z,UCgBA-PogWxh44p-lKZW9EVA,bébés ocelots et otocyons au Domaine des Fauves,"les bébés ocelots âgés de 7 semaines, arrivée ...",https://i.ytimg.com/vi/iHpgw8WN6Bg/default.jpg,120,...,True,True,False,5582,27,0,1,"<iframe width=""480"" height=""360"" src=""//www.yo...",[https://en.wikipedia.org/wiki/Lifestyle_(soci...,2011-10-31T00:00:00Z
2,youtube#video,R0zGKh5qNMofm0IBhZHvobnWkqM,XrX9NBMUOzw,j2Er,2006-08-05T18:57:44Z,UC4ONXuixSMf5o9O67A4gzhg,Two Giraffe Calves born at the San Francisco Zoo,Follow the zoo: \r\nhttp://twitter.com/sfzoo\r...,https://i.ytimg.com/vi/XrX9NBMUOzw/default.jpg,120,...,True,True,False,289627,245,0,59,"<iframe width=""480"" height=""360"" src=""//www.yo...",[https://en.wikipedia.org/wiki/Lifestyle_(soci...,2006-05-09T00:00:00Z
3,youtube#video,XY93dMczSpUYyP18PTAnGLAW7Ws,qwL5HKti-dw,3HOO,2009-12-03T14:46:09Z,UCWc9ziDlznO7v7cjzuT-Pqw,Fauna salbatica bucovineana in fotografi - Buk...,#WildBucovina Natura si fauna sălbatică din Bu...,https://i.ytimg.com/vi/qwL5HKti-dw/default.jpg,120,...,True,True,False,4456,23,0,5,"<iframe width=""480"" height=""360"" src=""//www.yo...","[https://en.wikipedia.org/wiki/Hobby, https://...",NaN
4,youtube#video,NcYca-t2DZKiXg5HhMYTuQeGBoI,wPJy9D_kzVU,6nfW,2010-07-13T06:23:18Z,UCtCCW9be1rUw_ig4UWsOSNg,Wyoming Archery Mule Deer Hunt With Joe Thomas...,,https://i.ytimg.com/vi/wPJy9D_kzVU/default.jpg,120,...,True,True,False,1338,2,0,0,"<iframe width=""480"" height=""270"" src=""//www.yo...","[https://en.wikipedia.org/wiki/Hobby, https://...",NaN


In [73]:
def get_videoComments(data, report=False):

    # for every yt8M id, check if comments is available
    # if it is, append to buffer list 
    
    no_comment_count = 0
    buffer = []
    for idx in tqdm(data.keys()):
        if data[idx]['comments'] == 'Invalid lookup':
            no_comment_count += 1
        elif len(data[idx]['comments']['items']) == 0:
            no_comment_count += 1
        else:
            data[idx]['comments']['items'][0]['yt8M_id'] = idx
            buffer.append(data[idx]['comments']['items'][0])
    
    # json_normalize buffer list into videoDetails df
    videoComs_df = pd.json_normalize(buffer, errors='ignore')
    
    # print stats for batch
    if report:
        print(f'{no_comment_count = }')

    return videoComs_df
    

In [74]:
test_df = get_videoComments(test_files, report=True)
test_df.head()

100%|████████████████████████████████████████| 15/15 [00:00<00:00, 31238.61it/s]

no_comment_count = 4


,kind,etag,id,yt8M_id,snippet.videoId,snippet.topLevelComment.kind,snippet.topLevelComment.etag,snippet.topLevelComment.id,snippet.topLevelComment.snippet.videoId,snippet.topLevelComment.snippet.textDisplay,...,snippet.topLevelComment.snippet.authorChannelUrl,snippet.topLevelComment.snippet.authorChannelId.value,snippet.topLevelComment.snippet.canRate,snippet.topLevelComment.snippet.viewerRating,snippet.topLevelComment.snippet.likeCount,snippet.topLevelComment.snippet.publishedAt,snippet.topLevelComment.snippet.updatedAt,snippet.canReply,snippet.totalReplyCount,snippet.isPublic
0,youtube#commentThread,EJSyRFx1dhqppMPt15RFmeUpnnM,UgjMGXaQw47fE3gCoAEC,HTft,130S1TN3P0M,youtube#comment,H5M32lmblp9WMe-eHLfBPKr_GrE,UgjMGXaQw47fE3gCoAEC,130S1TN3P0M,2:56 jeśli się nie mylę to pardwa:) Występuje ...,...,http://www.youtube.com/channel/UCJrxiBRv5IHv1U...,UCJrxiBRv5IHv1UJfKULEW7Q,True,none,0,2017-04-26T09:48:13Z,2017-04-26T09:48:13Z,True,2,True
1,youtube#commentThread,Oh-NTWh_vFipTc-3BKBTVz0LTEs,UgxLCQCLsca5zhKol2h4AaABAg,VX3C,iHpgw8WN6Bg,youtube#comment,cTsnqv7CcJvgJ0pznTYOoEt7vwY,UgxLCQCLsca5zhKol2h4AaABAg,iHpgw8WN6Bg,<3<3<3<3<3<3\nTrop ADORABLES!,...,http://www.youtube.com/channel/UCkHRUDRkrJPUXQ...,UCkHRUDRkrJPUXQfar1hChOg,True,none,0,2012-03-12T14:27:16Z,2012-03-12T14:27:16Z,True,0,True
2,youtube#commentThread,KHGlSgVVApoRzxY9_nMqbYTXWtE,UgySkF4KV4tbia-wGAF4AaABAg,j2Er,XrX9NBMUOzw,youtube#comment,pMET6ORoowLzZavBMrKFS1dpZPg,UgySkF4KV4tbia-wGAF4AaABAg,XrX9NBMUOzw,I loved how momma tapped at the calf to get th...,...,http://www.youtube.com/channel/UCc3BnE0KTjJzf6...,UCc3BnE0KTjJzf621gdN8L-A,True,none,0,2008-02-18T05:44:42Z,2008-02-18T05:44:42Z,True,0,True
3,youtube#commentThread,sXQJ7lTCShzrGvoqvf-p7OkJKA4,UgyccrM8fmN9z7gpArF4AaABAg,3HOO,qwL5HKti-dw,youtube#comment,WO9MkbKm5F3-XdRnDzJ3tnVPeDQ,UgyccrM8fmN9z7gpArF4AaABAg,qwL5HKti-dw,"Dacă ți-a plăcut, dă un ""LIKE"" și un ""SHARE"", ...",...,http://www.youtube.com/channel/UCWc9ziDlznO7v7...,UCWc9ziDlznO7v7cjzuT-Pqw,True,none,0,2022-07-08T12:56:11Z,2022-07-08T12:56:11Z,True,0,True
4,youtube#commentThread,Bm7jY1bz8gU1S1tt2BVeF26uE1Y,UgwQDLyNGIReuoaqvS94AaABAg,NX6e,S-MjHWCF_Cw,youtube#comment,pwQZ8vjRu669O7VLPjr4HvACbQE,UgwQDLyNGIReuoaqvS94AaABAg,S-MjHWCF_Cw,One day I'll decide to harvest!!! Thanks Rober...,...,http://www.youtube.com/channel/UCu_YcCDBu_UCmX...,UCu_YcCDBu_UCmXf2nmpw3vg,True,none,0,2009-11-24T12:18:59Z,2009-11-24T12:18:59Z,True,0,True


In [65]:
# read in batch1

batch_1 = '../data/raw/api_data/batch_1.txt'

with open(batch_1) as b1:
    b1_videos = json.load(b1)

In [66]:
# get videoDetails and Comments
vDetails_df = get_videoDetails(b1_videos, report=True)
vDetails_df.head()

100%|██████████████████████████████████████| 401/401 [00:00<00:00, 54138.34it/s]

n of videos successful: 393
n of videos failed: 8


,kind,etag,id,yt8M_id,snippet.publishedAt,snippet.channelId,snippet.title,snippet.description,snippet.thumbnails.default.url,snippet.thumbnails.default.width,...,recordingDetails.locationDescription,recordingDetails.location.latitude,recordingDetails.location.longitude,recordingDetails.location.altitude,snippet.defaultLanguage,localizations.en-US.title,localizations.en-US.description,contentDetails.regionRestriction.blocked,contentDetails.contentRating.ytRating,contentDetails.regionRestriction.allowed
0,youtube#video,qw6REl1mcyQq7UoW1-Dkm1yCrOY,130S1TN3P0M,HTft,2014-11-05T07:49:51Z,UCh_KdtEAc7qwyeXyVuuYPUA,Obserwator 22: Jesień na tamie / Autumn on a ...,To już pożegnanie z tamą bobrową. Jesień zawit...,https://i.ytimg.com/vi/130S1TN3P0M/default.jpg,120,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,youtube#video,hBay_xFV_egFO5I_5lVHS8xcSi4,iHpgw8WN6Bg,VX3C,2011-11-01T06:43:23Z,UCgBA-PogWxh44p-lKZW9EVA,bébés ocelots et otocyons au Domaine des Fauves,"les bébés ocelots âgés de 7 semaines, arrivée ...",https://i.ytimg.com/vi/iHpgw8WN6Bg/default.jpg,120,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,youtube#video,R0zGKh5qNMofm0IBhZHvobnWkqM,XrX9NBMUOzw,j2Er,2006-08-05T18:57:44Z,UC4ONXuixSMf5o9O67A4gzhg,Two Giraffe Calves born at the San Francisco Zoo,Follow the zoo: \r\nhttp://twitter.com/sfzoo\r...,https://i.ytimg.com/vi/XrX9NBMUOzw/default.jpg,120,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,youtube#video,XY93dMczSpUYyP18PTAnGLAW7Ws,qwL5HKti-dw,3HOO,2009-12-03T14:46:09Z,UCWc9ziDlznO7v7cjzuT-Pqw,Fauna salbatica bucovineana in fotografi - Buk...,#WildBucovina Natura si fauna sălbatică din Bu...,https://i.ytimg.com/vi/qwL5HKti-dw/default.jpg,120,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,youtube#video,NcYca-t2DZKiXg5HhMYTuQeGBoI,wPJy9D_kzVU,6nfW,2010-07-13T06:23:18Z,UCtCCW9be1rUw_ig4UWsOSNg,Wyoming Archery Mule Deer Hunt With Joe Thomas...,,https://i.ytimg.com/vi/wPJy9D_kzVU/default.jpg,120,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [81]:
vComments_df = get_videoComments(b1_videos, report=True)
vComments_df.head()

100%|██████████████████████████████████████| 401/401 [00:00<00:00, 57112.84it/s]

no_comment_count = 106


,kind,etag,id,yt8M_id,snippet.videoId,snippet.topLevelComment.kind,snippet.topLevelComment.etag,snippet.topLevelComment.id,snippet.topLevelComment.snippet.videoId,snippet.topLevelComment.snippet.textDisplay,...,snippet.topLevelComment.snippet.authorChannelId.value,snippet.topLevelComment.snippet.canRate,snippet.topLevelComment.snippet.viewerRating,snippet.topLevelComment.snippet.likeCount,snippet.topLevelComment.snippet.publishedAt,snippet.topLevelComment.snippet.updatedAt,snippet.canReply,snippet.totalReplyCount,snippet.isPublic,snippet.topLevelComment.snippet.moderationStatus
0,youtube#commentThread,EJSyRFx1dhqppMPt15RFmeUpnnM,UgjMGXaQw47fE3gCoAEC,HTft,130S1TN3P0M,youtube#comment,H5M32lmblp9WMe-eHLfBPKr_GrE,UgjMGXaQw47fE3gCoAEC,130S1TN3P0M,2:56 jeśli się nie mylę to pardwa:) Występuje ...,...,UCJrxiBRv5IHv1UJfKULEW7Q,True,none,0,2017-04-26T09:48:13Z,2017-04-26T09:48:13Z,True,2,True,NaN
1,youtube#commentThread,Oh-NTWh_vFipTc-3BKBTVz0LTEs,UgxLCQCLsca5zhKol2h4AaABAg,VX3C,iHpgw8WN6Bg,youtube#comment,cTsnqv7CcJvgJ0pznTYOoEt7vwY,UgxLCQCLsca5zhKol2h4AaABAg,iHpgw8WN6Bg,<3<3<3<3<3<3\nTrop ADORABLES!,...,UCkHRUDRkrJPUXQfar1hChOg,True,none,0,2012-03-12T14:27:16Z,2012-03-12T14:27:16Z,True,0,True,NaN
2,youtube#commentThread,KHGlSgVVApoRzxY9_nMqbYTXWtE,UgySkF4KV4tbia-wGAF4AaABAg,j2Er,XrX9NBMUOzw,youtube#comment,pMET6ORoowLzZavBMrKFS1dpZPg,UgySkF4KV4tbia-wGAF4AaABAg,XrX9NBMUOzw,I loved how momma tapped at the calf to get th...,...,UCc3BnE0KTjJzf621gdN8L-A,True,none,0,2008-02-18T05:44:42Z,2008-02-18T05:44:42Z,True,0,True,NaN
3,youtube#commentThread,sXQJ7lTCShzrGvoqvf-p7OkJKA4,UgyccrM8fmN9z7gpArF4AaABAg,3HOO,qwL5HKti-dw,youtube#comment,WO9MkbKm5F3-XdRnDzJ3tnVPeDQ,UgyccrM8fmN9z7gpArF4AaABAg,qwL5HKti-dw,"Dacă ți-a plăcut, dă un ""LIKE"" și un ""SHARE"", ...",...,UCWc9ziDlznO7v7cjzuT-Pqw,True,none,0,2022-07-08T12:56:11Z,2022-07-08T12:56:11Z,True,0,True,NaN
4,youtube#commentThread,Bm7jY1bz8gU1S1tt2BVeF26uE1Y,UgwQDLyNGIReuoaqvS94AaABAg,NX6e,S-MjHWCF_Cw,youtube#comment,pwQZ8vjRu669O7VLPjr4HvACbQE,UgwQDLyNGIReuoaqvS94AaABAg,S-MjHWCF_Cw,One day I'll decide to harvest!!! Thanks Rober...,...,UCu_YcCDBu_UCmXf2nmpw3vg,True,none,0,2009-11-24T12:18:59Z,2009-11-24T12:18:59Z,True,0,True,NaN


In [77]:
with open('../data/processed/batch1_videoDetails', 'wb') as f:
    pickle.dump(vDetails_df, f)

In [82]:
with open('../data/processed/batch1_videoComments', 'wb') as f:
    pickle.dump(vComments_df, f)